## Build simple travel brocure using LLM

### Plan
In this exercise, we are going to scrape the content of a site of a popular tourist spot. Then we are going to call LLM to summarize that content and build a simple travel brocure of that place. The travel brocure should be personalized based on the user preference given through the prompt.

### We will use the following LLMs and do a comparison
* Commercial
    * OpenAI GPT
    * Gemini
* Open Source
    * llama (from Meta)
    * DeepSeek

### Implementation - to be carried out through the following steps.

#### Step 1 - Import necessary libraries 

In [1]:
# To load environment variables
import os
from dotenv import load_dotenv 

# To call the OpenAI library which can call any commercial or open-source LLMs
from openai import OpenAI

# To display the output in nice format instead of plain text
from IPython.display import Markdown, display

# To scrape the content from a website
from bs4 import BeautifulSoup
import requests

#### Step 2 - Create the function to scrape content from site
Courtesy to Ed Donner for building this awesome function as part of the LLM Engineering course. I have taken that code and only made few minor changes.

In [9]:
# Standard headers to fetch a website
from bs4.element import Tag


header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# Web scraper function
def fetch_website_contents(url):
    """
    Return the title and 10,000 character lengtch of contents of the given website url;
    """
    
    response = requests.get(url, headers=header)
    soup = BeautifulSoup(response.content, "html.parser")
    site_title = soup.title.string if soup.title else "No title found"
    if soup.body:
        # The following loop 
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        site_content = soup.body.get_text(separator="\n", strip=True)
    else:
        site_content = ""
    return (site_title + "\n\n" + site_content)[:20000]

#### Step 3 - Call the web scraper function to scrape content a website
We will scrape the content of a travel page about [Barcelona](https://www.britannica.com/place/Barcelona) created by [Britannica](https://www.britannica.com/).

In [ ]:
# Initialize the url of the site
url = "https://www.britannica.com/place/Barcelona"

# Call the scraper function to display the content
site_content = fetch_website_contents(url=url)

# Display the content
site_content

#### Step 4 - Load and validate the API keys of the LLMs 


In [ ]:
# Load the API keys of the LLMs stored as environment variables in a file called .env

load_dotenv(override=True)
# OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')
# Google API key 
google_api_key = os.getenv("GOOGLE_API_KEY")

# Validate the OpenAI API key
if not openai_api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not openai_api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif openai_api_key.strip() != openai_api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("OpenAI API key found and looks good so far!")


# Validate the Google API key
if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("Google API key found and looks good so far!")

#### Step 5 - Define the system and user prompt
**System prompt** is how we instruct the LLM as we are building the agent. This is where we can set the high level rules and the guardrails.

**User prompt** is how the user of the agent sends instruction to the agent through prompt.

In [14]:
# Define our system prompt
system_prompt = """
You are a helpful assistant that analyzes the contents of a website, 
and generate a travel brochure customized for the user based on the 
information provided in the user prompt. Respond in markdown.
"""

# Define our user prompt
user_prompt_prefix = """
Hi there, my name is Ashim. I am interested to visit Barcelona with my wife. 
Here are the contents of a website, where I found lots of information.
I need your help to know better about the place. Thank you.
"""

# Create a dictionary with the system and user prompt to be fed to the LLM
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt_prefix + site_content}
]


#### Step 6a.1 - Generate travel brochure of Barcelona using OpenAI gpt-4.1-nano model

In [15]:
# Instantiate the OpenAI using the API key
openai = OpenAI(api_key=openai_api_key)

# Generate the response using the model
response = openai.chat.completions.create(model="gpt-4.1-nano", messages=messages)

# Display the response from the LLM
display(Markdown(response.choices[0].message.content))

# Discover the Vibrant Charm of Barcelona – Your Perfect Travel Destination

Welcome, Ashim and your wife! Embark on an unforgettable journey to Barcelona, a city rich in history, culture, and breathtaking sights. This travel brochure highlights all you need to know to make your trip extraordinary.

---

## Why Choose Barcelona?

- **Cultural Capital**: Known as the "Paris of Spain," Barcelona boasts a lively arts scene, stunning architecture, and a deep-rooted history.
- **Ideal Climate**: Enjoy mild temperatures averaging 61°F (16°C) with warm summers, perfect for exploring year-round.
- **Beautiful Scenery**: Nestled between scenic mountains and the Mediterranean Sea, the city offers spectacular landscapes and beaches.

---

## Highlights of Barcelona

### Architectural Marvels
- **Sagrada Família**: Antoni Gaudí's masterpiece, an iconic and elaborately designed basilica still under construction, symbolizes the city's artistic spirit.  
- **Modernist Landmarks**: Casa Batlló, Casa Milà (La Pedrera), and Güell Park showcase innovative architecture declared UNESCO World Heritage sites.
- **Gothic Quarter**: Wander through narrow medieval streets, marvel at historical buildings, cathedral, and medieval walls.

### Beautiful Promenades & City Life
- **Las Ramblas**: The lively boulevard filled with stalls, flower kiosks, cafes, street performers, and vibrant energy leading to the port.
- **Port & Waterfront**: Relax at the marina, enjoy fresh seafood, or take a cruise along the Mediterranean.

### Cultural & Historical Attractions
- **Palace of the Generalitat & Royal Palace**: Witness Barcelona’s importance as a political and religious hub.
- **Montjuïc Hill**: Offers panoramic views, parks, and historic sites like the Olympic stadium.

---

## Climate & Best Time to Visit
- **Spring & Fall**: Mild and pleasant weather, ideal for sightseeing.
- **Summer**: Warm (up to 76°F/24°C), perfect for beach days and festivals.
- **Winters**: Cooler but still mild, with temperatures around 49°F (9°C).

---

## Getting Around
- Efficient public transportation including buses, trams, and the metro.
- Walkable city centers, especially around La Rambla and Gothic Quarter.
- Biking options for scenic exploration.

---

## Experience Barcelona's Unique Culture
- **Festivals & Events**: Celebrations like La Mercè Festival and human towers (Castells).
- **Cuisine**: Savor tapas, seafood, and traditional Catalan dishes in local taverns.
- **Languages**: Catalan and Spanish are spoken; English is widely understood in tourist areas.

---

## Practical Tips
- **Stay** in centrally located districts like Eixample or Gothic Quarter for easy access to attractions.
- **Book** tickets in advance for popular sites like Sagrada Família to avoid queues.
- **Dress** comfortably and be prepared for sunny days with sunscreen and light clothing.

---

## Your Next Adventure Awaits!
Discover the enchanting blend of history, art, and natural beauty in Barcelona. Whether lounging on beaches, admiring Gaudí’s genius, or strolling through medieval streets, this city promises an inspiring experience for you and your wife.

---

# Bon Voyage, Ashim! Explore Barcelona – A City of Infinite Possibilities!

#### Step 6a.2 - Generate travel brochure of Barcelona using Google gemini-2.5-flash-lite model

In [17]:
# Note we don't explicitely mention base_url while calling OpenAI for OpenAI API,
# because in that case we are using the default base_url, which is "https://api.openai.com/v1"
# However, to call non OpenAI LLMs, we need to provide the endpoint of the model.
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

# Instantiate the OpenAI using the API key
gemini = OpenAI(
    api_key=google_api_key,
    base_url=GEMINI_BASE_URL
)

# Generate the response using the model
response = gemini.chat.completions.create(model="gemini-2.5-flash-lite", messages=messages)

# Display the response from the LLM
display(Markdown(response.choices[0].message.content))

## Discover Barcelona: A Romantic Escape for You and Your Wife!

Ashim, get ready to be captivated by Barcelona, a city that beautifully blends rich history, vibrant culture, and stunning Mediterranean charm. This guide, inspired by the wealth of information from Britannica, is designed to help you and your wife experience the very best of this extraordinary city.

### Why Barcelona for Your Getaway?

Barcelona, often called the "Paris of Spain," offers a unique and unforgettable experience. It's a city renowned for its individuality, cultural depth, and breathtaking beauty. The blend of scenic mountains, the sparkling Mediterranean Sea, and a climate that invites you to stroll its streets creates an atmosphere perfect for romance and discovery.

### A Glimpse into Barcelona's Rich Tapestry:

*   **A Storied Past:** Barcelona boasts a history that stretches back to Roman times. Explore the narrow, winding streets of the **Gothic Quarter**, the ancient heart of the city, where medieval buildings, churches, and palaces whisper tales of centuries past. You can even see remnants of Roman walls incorporated into the city's fabric!

*   **Architectural Wonders:** Prepare to be amazed by Barcelona's architectural prowess. The city is a treasure trove of **Modernist and Art Nouveau** masterpieces.
    *   **Antoni Gaudí's** works are simply mesmerizing. The iconic **Sagrada Família** is an absolute must-see, a colossal temple that has become a symbol of the city. Don't miss his other incredible creations like **Casa Batlló, Casa Milà (La Pedrera), and Güell Park**, all designated as UNESCO World Heritage sites.
    *   Discover the stunning **Music Palace**, another UNESCO World Heritage site, showcasing the remarkable talent of architects like Luis Doménechi Montaner.

*   **The Enchanting Ramblas:** Stroll hand-in-hand down **Las Ramblas**, Barcelona's most famous promenade. This lively boulevard, lined with trees, is a delightful spectacle of flower stalls, kiosks selling books and newspapers, and a vibrant street life that leads you down to the port and the impressive Columbus Monument.

*   **A City of Contrasts:** From its ancient core to the expansive **L'Eixample** district, designed with geometric blocks and open spaces, Barcelona showcases a fascinating evolution. While modern buildings add to the skyline, the city's soul lies in its historical layers and the artistic flair that permeates its streets.

### Practical Delights for Your Trip:

*   **Climate:** Enjoy Barcelona's **benign climate**. With average annual temperatures of 61°F (16°C), it's pleasant year-round. Winters are mild (averaging 49°F/9°C in January), and summers are warm (averaging 76°F/24°C in August). Pack accordingly for comfortable exploration!

*   **Location:** Nestled between scenic mountains and the Mediterranean Sea in northeastern Spain, Barcelona offers a stunning natural setting. Its strategic location has shaped its history and continues to draw visitors from around the globe.

### Cultural Vibrancy and Modern Life:

*   **Catalan Culture:** Barcelona is the heart of Catalonia, and its unique **Catalan culture** is celebrated with pride. The official recognition of the Catalan language and regional self-government has revitalized its cultural scene, offering a deep and diverse experience for visitors.

*   **Economic Hub:** As Spain's major Mediterranean port and commercial center, Barcelona buzzes with energy. This vitality is evident in its bustling markets, lively streets, and thriving economy.

### Planning Your Visit:

While this information provides a wonderful overview, we encourage you to explore the official tourism websites for Barcelona to find specific recommendations on:

*   **Accommodation:** From charming boutique hotels to luxurious stays, Barcelona offers a wide range of options.
*   **Dining:** Indulge in delicious Catalan cuisine, from fresh seafood to traditional tapas.
*   **Activities:** Discover museums, galleries, live music venues, and much more.
*   **Transportation:** Navigate the city with ease using its efficient public transport system.

Ashim, Barcelona promises a journey filled with beauty, history, and romance for you and your wife. Get ready to create lasting memories in this captivating Spanish gem!

### Comparison between Gemini 2.5 Flash Lite vs GPT-4.1 Nano

* **Performance:** Gemini 2.5 Flash Lite leads with a 16.2% higher average benchmark score, making it a stronger choice for coding tasks. The same reflects from the above use-case, where the GPT-4.1 Nano took
8.8 seconds to gererate the travel brochure, whereas it took 6.1 seconds for Gemini 2.5 Flash Lite.
* **Context Window:** Gemini 2.5 Flash Lite offers 33.8K more tokens in the context window than GPT-4.1 Nano, which can be beneficial for more complex tasks requiring extensive context. 
* **Pricing:** Both models have similar pricing, with GPT-4.1 Nano being about 1.5 times cheaper overall. 

In summary, Gemini 2.5 Flash Lite is the more advanced model with better performance and context capabilities, while GPT-4.1 Nano is a cost-effective option with similar performance metrics. The choice between the two will depend on the specific requirements of the application and the budget considerations.

#### Step 6b.1 - Generate travel brochure of Barcelona using the open-source *llama3.2* model running locally

In [ ]:
# Check and make sure ollama is running
requests.get("http://localhost:11434").content

b'Ollama is running'

In [ ]:
# Note we don't explicitely mention base_url while calling OpenAI for OpenAI API,
# because in that case we are using the default base_url, which is "https://api.openai.com/v1"
# However, to call non OpenAI LLMs, we need to provide the endpoint of the model.
OLLAMA_BASE_URL = "http://localhost:11434/v1" 

# Instantiate the OpenAI using the API key Though the api_key is not relevant as the model 
# is running locally, still as the parameter is mandatory, so we need to pass any string. 
ollama = OpenAI(
    api_key="NA",
    base_url=OLLAMA_BASE_URL
)

# Generate the response using the llama3.2 model
response = ollama.chat.completions.create(model="llama3.2", messages=messages)

# Display the response from the LLM
display(Markdown(response.choices[0].message.content))

**Travel Brochure for Barcelona**

Welcome to Barcelona, a vibrant and eclectic city that combines stunning architecture, rich history, and breathtaking natural beauty.

**Introduction**

Barcelona is situated on a plain along the Mediterranean Sea, with a landscape dominated by mountains and a few scenic valleys. The city lies at an altitude of about 100 meters (330 feet) above sea level. This unique topography has influenced its history, urban planning and development over time.

**Must-See Attractions**

1.  **Barceloneta Beach**: Enjoy stunning views of the Mediterranean Sea from this popular beach.
2.  **Park Güell**: Explore the iconic park designed by Antoni Gaudí featuring public art installations and city views.
3.  **Cathedral of Barcelona**
4.  **La Rambla:** Wander through this famous street lined with shops, cafes, and cultural attractions.
5.  **Palau de la Música Catalana**: Discover the stunning architecture inside and out which combines medieval style.
6.  **Tapas Bars:** Experience local cuisine with a focus on traditional dishes and drinks.

**Cultural Highlights**

*   Visit museums showcasing world-class art collections
*   Catch a show or concert in one of the many theaters
*   Enjoy street performers and buskers in various areas along la Rambla.
*   Explore Barcelona's vibrant markets and shopping spots

**Getting Around**

Barcelona offers an excellent public transportation system. From modern buses to historic tramways, there are several options for getting around.

*   Visit the port area on foot or by taking a ferry across
*   Discover hidden gems of local cuisine at street food destinations like stalls near the beaches.
*   Experience a wide variety of accommodation options ranging from simple apartments.

**Accommodation Options**

From budget-friendly guesthouses to luxurious hotels, there are plenty of choices:

1.  **Budget-Friendly Option**: Stay with guests in modern flats and hostels.
2.  **Mid-Range Hotels**: Affordable yet comfortable, perfect for exploring at leisure.
3.  **Luxury Accommodations**: Experience top-notch services, views, and convenience.

**Insider Tips and Tricks**

Enjoying a meal can be challenging but consider the food from some local markets like La Boqueria Market 

Explore hidden destinations during your stay

Always research which attractions fit best based on your interests and preferences

#### Step 6b.2 - Generate travel brochure of Barcelona using the open-source *deepseek-r1:1.5b* model running locally

In [21]:
# Note we don't explicitely mention base_url while calling OpenAI for OpenAI API,
# because in that case we are using the default base_url, which is "https://api.openai.com/v1"
# However, to call non OpenAI LLMs, we need to provide the endpoint of the model.
OLLAMA_BASE_URL = "http://localhost:11434/v1" 

# Instantiate the OpenAI using the API key Though the api_key is not relevant as the model 
# is running locally, still as the parameter is mandatory, so we need to pass any string. 
ollama = OpenAI(
    api_key="NA",
    base_url=OLLAMA_BASE_URL
)

# Generate the response using the deepseek-r1:1.5b model
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=messages)

# Display the response from the LLM
display(Markdown(response.choices[0].message.content))

**Imagining Travel Through Barcelona: A Strategic Guide**

---

### **Imagining Structure**
To effectively navigate the world of Barcelona, consider an image guide where related information is clustered:

1. **Urban Connectors**: Include key images that bridge major cities (e.g., Gomini and Barcelona in Valencian).
2. **Transportation Mixtures**: Place a mix photo of metro, buses, and other modes together.
3. **Cultural Web**: Link city landmarks side by side with related venues.
4. **Festivals and Events**: Use images alongside their cities for immediate connection.

---

### **Key Image Placement Suggestions**
1. **Major Cities/Transportations**: Combine image tags of related cities or transportation modes in visual guides.
2. **Cultural Attractions**: Place images side-by-side, such as city landmarks and Gaudí's works.
3. **Event Frenetic Flavors**: Position event visuals next to significant venues in series.

---

### **Up-to-Date Information**
- **Events in 2024**: Highlight Barcelona’s COVID response with photos from March.
- **Exploring Spanish/Turkish Culture**: Update links to blogs on Turkish food and vibrant art exhibitions.

---

### **Travel Guide Links**
For practical access, explore:
1. Visit [Barcelona Explorades](https://www.britannica.com/places/barcelona) for detailed facts.
2. Check out [Valencian travel tips](https://www-valenciamuse.fr/) for Valencian travelers.
3. Utilize local blogs for unique tours and reviews.

---

This structured approach will enhance reader engagement, aiding in effective travel planning with engaging visuals and connections.

### Comparison between the Commercial (Gemini 2.5 Flash Lite, or GPT-4.1 Nano) and Open-Source (llama3.2, or deepseek-r1:1.5b) models

* **Execution Speed:** Both commercial models performed way faster than the open-source models 
    * GPT-4.1 Nano took **8.8** seconds
    * Gemini 2.5 Flash Lite took **6.3** seconds
    * Llama3.2 took **271.6** seconds
    * Deepseek-r1:1.5b took **144.5** seconds
* **Result Quality:** The quality of the travel brochure generated by both commercial models are comparatively better and more personalized with respect to those generated by the open-source models. 
* **Pricing:** The both open-source models win in this category because we can run them free locally on our computer. 